# Local Plan Reference Column Reassignment

## Overview
This notebook reassigns reference columns for local plan data to improve consistency and clarity across datasets.

## Purpose
The primary goals are to:
1. **Create standardised references** for local plans following a consistent naming format
2. **Handle joint local plans** differently by using the joint plan name instead of individual authority names
3. **Apply these new references** across related datasets (local plan documents and timetables)
4. **Resolve data quality issues** where document references don't match their source local plan references

## Workflow

### 1. Reference Creation (`reference_func`)
Generates new reference columns (`reference2`) for the main local plan dataset (`lp`) with the format:
- **Single local plans**: `{organisation-slug}-local-plan-{year}`
  - Example: `epping-forest-district-council-local-plan-2011`
- **Joint local plans**: `{joint-plan-name-slug}` (no year suffix)
  - Example: `the-babergh-and-mid-suffolk-joint-local-plan`

Joint plans are identified by the presence of a semicolon (`;`) in the `organisations` column.

### 2. Lookup Generation
Creates mapping dictionaries from old references to new references:
- `ref_lookup`: Maps old local plan references to new ones
- `ref_lookup_documents`: Maps old document references to new ones
- `ref_lookup_timetable`: Maps old timetable references to new ones

### 3. Application to Related Datasets
Applies the new references to the `lp_doc` (local plan documents) dataset by:
- Matching documents to their source local plan using the `local-plan` column
- Looking up the new reference from the mapping
- Handling duplicates by adding unique suffixes (`-1`, `-2`, `-3`, etc.)

### 4. Data Quality Resolution
Identifies and fixes mismatches where document `local-plan` values don't exist in the source data, ensuring all references resolve correctly.

## Output
A consolidated lookup dataframe combining all three mapping dictionaries for use in updating external systems.

In [1]:
# Import libraries
import pandas as pd
import numpy as np

In [2]:
# Read in source data to be changed
lp = pd.read_csv('source-data/local-plan.csv')
lp_doc = pd.read_csv('source-data/local-plan-document.csv')
lp_time = pd.read_csv('source-data/local-plan-timetable.csv')
lp_bound = pd.read_csv('source-data/local-plan-boundary.csv')
lookup = pd.read_csv('source-data/lookup.csv')

# Read in LPA name lookup
lpa_lookup = pd.read_csv('source-data/lpa-lookup.csv')
org_lookup = pd.read_csv('source-data/organisation.csv')

## Local plans

In [3]:
# Map organisation names
lp['organisation-name'] = lp['organisations'].map(
    lpa_lookup.set_index('organisation')['organisation_label']
)

# Replace missing Lincolnshire County Council name
lp.loc[lp['organisations'] == 'local-authority:LIN', 'organisation-name'] = 'Lincolnshire County Council'

In [4]:
def reference_func(source_df, lpa_col_name):
    """
    Create a reference column for each row in the dataframe.
    Format: {organisation-slug}-local-plan-{year}
    
    For joint local plans (organisations column contains ';'), uses slugified 'name' column only.
    Otherwise uses {lpa-slug}-local-plan-{year}.
    
    Uses period-start-date for year, falls back to period-end-date if NaN,
    then falls back to empty string if both are NaN.
    
    Args:
        source_df: DataFrame with 'organisation-name', 'period-start-date', 
                   and 'period-end-date' columns
    
    Returns:
        DataFrame with new 'reference' column
    """
    df = source_df.copy()
    
    # Check if it's a joint local plan (organisations contains ';')
    is_joint = df['organisations'].str.contains(';', na=False)
    
    # Create slug for non-joint (from lpa_col_name)
    regular_slug = (
        df[lpa_col_name]
        .str.replace(' ', '-')
        .str.lower()
    )
    
    # Create slug for joint (from 'name')
    joint_slug = (
        df['name']
        .str.replace(' ', '-')
        .str.lower()
    )
    
    # Extract year: try period-start-date first, fallback to period-end-date, then ''
    year = df['period-start-date'].fillna(df['period-end-date']).fillna('').astype(str)
    year = year.str.replace('.0', '')
    
    # Generate reference
    # Joint plans: just the name slug
    # Non-joint: {slug}-local-plan-{year}
    regular_reference = regular_slug + '-local-plan-' + year
    regular_reference = regular_reference.str.rstrip('-')
    
    df['reference2'] = np.where(is_joint, joint_slug, regular_reference)
    
    return df

In [5]:
# Create new reference column in local-plans
lp = reference_func(lp, lpa_col_name='organisation-name')

In [6]:
# Create a lookup of old to new references
ref_lookup = lp.set_index('reference')['reference2'].to_dict()

## Documents

In [7]:
# Update the Cambridge row to use the correct local-plan reference
lp_doc.loc[lp_doc['reference'] == 'cambridge-local-plan', 'local-plan'] = 'cambridge-city-council-local-plan-2018'

# Map the local-plan references in lp_doc to the new reference2 values
lp_doc['reference2'] = lp_doc['local-plan'].map(ref_lookup)

# Add numbers to the end of the reference2 columns
# Identify which reference2 values are duplicated
counts = lp_doc['reference2'].value_counts()
duplicated_refs = counts[counts > 1].index

# Get cumulative count within each group (starting from 1)
counter = (lp_doc.groupby('reference2').cumcount() + 1).fillna(0).astype(int)

# Add suffix only to duplicates
mask = lp_doc['reference2'].isin(duplicated_refs)
lp_doc['reference2'] = lp_doc['reference2'].where(~mask, lp_doc['reference2'] + '-' + counter.astype(str))

In [8]:
# Create a lookup of old to new references for documents
ref_lookup_documents = lp_doc.set_index('reference')['reference2'].to_dict()

In [9]:
# Find which local-plan values don't have a mapping
missing_mappings = lp_doc[lp_doc['reference2'].isna()]['local-plan'].unique()
print("Local plans with no mapping:")
print(missing_mappings)

# Check if they exist in the lp dataframe
for lp_ref in missing_mappings:
    exists_in_lp = lp[lp['reference'] == lp_ref].shape[0] > 0
    print(f"  {lp_ref}: {exists_in_lp}")

Local plans with no mapping:
[]


## Timetables

In [10]:
# Map the local-plan references in lp_doc to the new reference2 values
lp_time['reference2'] = lp_time['local-plan'].map(ref_lookup) + '-' + lp_time['local-plan-event']

# Add numbers to the end of the reference2 columns
# Identify which reference2 values are duplicated
counts = lp_time['reference2'].value_counts()
duplicated_refs = counts[counts > 1].index

# Get cumulative count within each group (starting from 1)
counter = (lp_time.groupby('reference2').cumcount() + 1).fillna(0).astype(int)

# Add suffix only to duplicates
mask = lp_time['reference2'].isin(duplicated_refs)
lp_time['reference2'] = lp_time['reference2'].where(~mask, lp_time['reference2'] + '-' + counter.astype(str))

In [11]:
# Check if there are any more duplicates (returns either True or False)
lp_time['reference2'].duplicated().any()

np.False_

In [12]:
# Create a lookup of old to new references for timetables
ref_lookup_timetable = lp_time.set_index('reference')['reference2'].to_dict()

## Boundaries

In [13]:
# Load the mapping from organisation.csv
lad_to_lpa = dict(zip(org_lookup['local-authority-district'], org_lookup['local-planning-authority']))

# Map LAD codes to LPA codes
lp_bound['reference2'] = lp_bound['reference'].map(lad_to_lpa)

In [14]:
# Create a lookup of old to new references for boundaries
ref_lookup_boundary = lp_bound.set_index('reference')['reference2'].to_dict()

## Collate all the lookups together

In [15]:
# Combine all three dictionaries
combined_lookup = ref_lookup | ref_lookup_documents | ref_lookup_timetable | ref_lookup_boundary

# Convert into a DataFrame
combined_df = pd.Series(combined_lookup).reset_index()
combined_df.columns = ['reference', 'new_reference']


In [16]:
combined_df

,reference,new_reference
0,london-borough-of-haringey-local-plan-2013,london-borough-of-haringey-local-plan-2013
1,new-canterbury-district-local-plan-2020-2040,canterbury-city-council-local-plan-2020
2,east-cambridgeshire-district-council-local-pla...,east-cambridgeshire-district-council-local-pla...
3,east-cambridgeshire-local-plan-single-issue-re...,east-cambridgeshire-district-council-local-pla...
4,london-borough-of-sutton-local-plan-2018,london-borough-of-sutton-local-plan-2016
...,...,...
1154,E06000056,E60000144
1155,E07000079,E60000309
1156,E08000009,E60000033
1157,E07000082,E60000312


## Update existing lookup with new references

In [17]:
# Merge the old lookup with the new references
new_lookup = pd.merge(lookup, combined_df, on='reference', how='left')

# Update the reference column with new references where available (otherwise keep old, e.g. boundary refs)
new_lookup['reference'] = np.where(new_lookup['new_reference'].notna(), new_lookup['new_reference'], new_lookup['reference'])

In [18]:
# Save updated lookup to CSV
new_lookup.drop(columns=['new_reference']).to_csv('fixed-data/lookup.csv', index=False)